In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F
from pyspark.ml import feature as MF
from pyspark.ml.stat import Correlation

spark = SparkSession.builder \
    .appName("SparkReadOperations") \
    .config("spark.executor.cores", "12") \
    .config("spark.executor.memory", "48g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "12") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [ ]:
data = spark.read.csv('Loan_Default.csv',header = True)
data.show()

+-----+----+----------+-----------------+-------------+---------+------------+-----------------+-----------+----------------------+-----------+----------------+--------------------+---------------+-----+-----------------+-------------+----------------+--------------+-----------------+--------------+----------+-----------+-------+-----------+------------+------------------------+-----+-------------------------+-----------+-------+-------------+------+-----+
|   ID|year|loan_limit|           Gender|approv_in_adv|loan_type|loan_purpose|Credit_Worthiness|open_credit|business_or_commercial|loan_amount|rate_of_interest|Interest_rate_spread|Upfront_charges| term|Neg_ammortization|interest_only|lump_sum_payment|property_value|construction_type|occupancy_type|Secured_by|total_units| income|credit_type|Credit_Score|co-applicant_credit_type|  age|submission_of_application|        LTV| Region|Security_Type|Status|dtir1|
+-----+----+----------+-----------------+-------------+---------+------------+

In [ ]:
new_data = data.drop('ID','year')

In [ ]:
new_data = new_data.withColumn('Age_range_1',F.split(F.col('age'),'-')[0]).withColumn('Age_range_2',F.split(F.col('age'),'-')[1])
new_data= new_data.drop('age')

In [ ]:
new_data.createOrReplaceTempView('df')
col_to_double = ['loan_amount','rate_of_interest','Interest_rate_spread','Upfront_charges','term','income','dtir1','Status','LTV','Credit_Score','property_value','Age_range_1','Age_range_2']
col_to_string = list(set(new_data.columns) - set(col_to_double))
query = ", ".join([f"CAST({col} AS DOUBLE) AS {col}" for col in col_to_double if col])

query1 = ", ".join(f"CAST(`{col}` AS STRING) AS `{col}`" for col in col_to_string if col)

df_new = spark.sql(f"SELECT {query}, {query1} FROM df")

In [ ]:
df_new.createOrReplaceTempView('df')

In [ ]:
total = data.count()
details = spark.createDataFrame([

  Row(column_name = col,
      type_of_data = df_new.dtypes[i][1],
      null = spark.sql(f'select `{col}` from df where `{col}` is NULL').count(),
        null_pct = spark.sql(f'select `{col}` from df where `{col}` is NULL').count()*100// total,
        unique_value = spark.sql(f'select Distinct(`{col}` )from df').count()

  )
  for i,col in enumerate(df_new.columns)
])
details.show(35,truncate=False)

+-------------------------+------------+-----+--------+------------+
|column_name              |type_of_data|null |null_pct|unique_value|
+-------------------------+------------+-----+--------+------------+
|loan_amount              |double      |0    |0       |211         |
|rate_of_interest         |double      |36439|24      |132         |
|Interest_rate_spread     |double      |36639|24      |22517       |
|Upfront_charges          |double      |39642|26      |58272       |
|term                     |double      |41   |0       |27          |
|income                   |double      |9150 |6       |1002        |
|dtir1                    |double      |24121|16      |58          |
|Status                   |double      |0    |0       |2           |
|LTV                      |double      |15098|10      |8485        |
|Credit_Score             |double      |0    |0       |401         |
|property_value           |double      |15098|10      |386         |
|Age_range_1              |double 

## handeling missing values


In [ ]:
df_new = spark.sql(f'Select *, Coalesce(rate_of_interest, 0) as rate_of_interest_,  Coalesce(Interest_rate_spread, 0) as Interest_rate_spread_, \
Coalesce(Upfront_charges, 0) as Upfront_charges_ , Coalesce(dtir1, 0) as dtir1_ from df')
df_new = df_new.drop('rate_of_interest','Interest_rate_spread','Upfront_charges','dtir1')

In [ ]:
df_new.show()

+-----------+-----+-------+------+-----------+------------+--------------+-----------+-----------+-----------+-------------+-------------------------+-----------------+-----------+-------------+----------------+-----------+----------+-----------------+----------------------+-------+-----------------+--------------+-------------+------------------------+----------+-----------------+---------+------------+-----------------+---------------------+----------------+------+
|loan_amount| term| income|Status|        LTV|Credit_Score|property_value|Age_range_1|Age_range_2|open_credit|interest_only|submission_of_application|Credit_Worthiness|credit_type|approv_in_adv|lump_sum_payment|total_units|Secured_by|Neg_ammortization|business_or_commercial| Region|           Gender|occupancy_type|Security_Type|co-applicant_credit_type|loan_limit|construction_type|loan_type|loan_purpose|rate_of_interest_|Interest_rate_spread_|Upfront_charges_|dtir1_|
+-----------+-----+-------+------+-----------+----------

In [ ]:
df_new.createOrReplaceTempView('df')
total = data.count()

details = spark.createDataFrame([

  Row(column_name = col,
      type_of_data = df_new.dtypes[i][1],
      null = spark.sql(f'select `{col}` from df where `{col}` is NULL').count(),
        null_pct = spark.sql(f'select `{col}` from df where `{col}` is NULL').count()*100// total,
        unique_value = spark.sql(f'select Distinct(`{col}` )from df').count()

  )
  for i,col in enumerate(df_new.columns)
])
details.show(35,truncate=False)

+-------------------------+------------+-----+--------+------------+
|column_name              |type_of_data|null |null_pct|unique_value|
+-------------------------+------------+-----+--------+------------+
|loan_amount              |double      |0    |0       |211         |
|term                     |double      |41   |0       |27          |
|income                   |double      |9150 |6       |1002        |
|Status                   |double      |0    |0       |2           |
|LTV                      |double      |15098|10      |8485        |
|Credit_Score             |double      |0    |0       |401         |
|property_value           |double      |15098|10      |386         |
|Age_range_1              |double      |8712 |5       |6           |
|Age_range_2              |double      |8712 |5       |6           |
|open_credit              |string      |0    |0       |2           |
|interest_only            |string      |0    |0       |2           |
|submission_of_application|string 

In [ ]:
print("Before Drop Null value Data size: ",df_new.count())
df_new = df_new.dropna()
print("After Dropped Null value Data size: ",df_new.count())

Before Drop Null value Data size:  148670
After Dropped Null value Data size:  113874


In [ ]:
df_new.show()

+-----------+-----+-------+------+-----------+------------+--------------+-----------+-----------+-----------+-------------+-------------------------+-----------------+-----------+-------------+----------------+-----------+----------+-----------------+----------------------+-------+-----------------+--------------+-------------+------------------------+----------+-----------------+---------+------------+-----------------+---------------------+----------------+------+
|loan_amount| term| income|Status|        LTV|Credit_Score|property_value|Age_range_1|Age_range_2|open_credit|interest_only|submission_of_application|Credit_Worthiness|credit_type|approv_in_adv|lump_sum_payment|total_units|Secured_by|Neg_ammortization|business_or_commercial| Region|           Gender|occupancy_type|Security_Type|co-applicant_credit_type|loan_limit|construction_type|loan_type|loan_purpose|rate_of_interest_|Interest_rate_spread_|Upfront_charges_|dtir1_|
+-----------+-----+-------+------+-----------+----------

In [ ]:
print(col_to_string)

['open_credit', 'interest_only', 'submission_of_application', 'Credit_Worthiness', 'credit_type', 'approv_in_adv', 'lump_sum_payment', 'total_units', 'Secured_by', 'Neg_ammortization', 'business_or_commercial', 'Region', 'Gender', 'occupancy_type', 'Security_Type', 'co-applicant_credit_type', 'loan_limit', 'construction_type', 'loan_type', 'loan_purpose']


In [ ]:
df1 = df_new
col_to_number = col_to_string
string_columns_indexed = [f"{n[0]}_indexed" for n in df1.dtypes if n[1] == "string"]


from pyspark.ml import Pipeline

indexer = MF.StringIndexer(
    inputCols=col_to_number,
    outputCols=string_columns_indexed
)
indexer_model =indexer.fit(df1)
df_indexer = indexer_model.transform(df1)

# Drop string columns
df_indexer = df_indexer.drop(*col_to_number)
indexer_model.save("indexer_model")


In [ ]:
df_indexer.toPandas().to_csv('Cleaned_Loan_Data.csv',index=False)

In [ ]:
assembler_df = MF.VectorAssembler(
    inputCols=[c for c in df_indexer.columns if c != "Status"],
    outputCol="features"
).transform(df_indexer)
assembler_df.show(2)

+-----------+-----+------+------+-----------+------------+--------------+-----------+-----------+-----------------+---------------------+----------------+------+-------------------+---------------------+---------------------------------+-------------------------+-------------------+---------------------+------------------------+-------------------+------------------+-------------------------+------------------------------+--------------+--------------+----------------------+---------------------+--------------------------------+------------------+-------------------------+-----------------+--------------------+--------------------+
|loan_amount| term|income|Status|        LTV|Credit_Score|property_value|Age_range_1|Age_range_2|rate_of_interest_|Interest_rate_spread_|Upfront_charges_|dtir1_|open_credit_indexed|interest_only_indexed|submission_of_application_indexed|Credit_Worthiness_indexed|credit_type_indexed|approv_in_adv_indexed|lump_sum_payment_indexed|total_units_indexed|Secured_by

In [ ]:
main_data = assembler_df.select('features','Status')
main_data = main_data.withColumnRenamed('Status','label')

train_df, test_df = main_data.randomSplit([.75,.25])

In [ ]:
print('Train data size: ',train_df.count())
print('Test data size: ',test_df.count())

Train data size:  85416
Test data size:  28458


In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



In [ ]:
def train_data_func(data,test_data):
    lr = LogisticRegression()
    model = lr.fit(data)
    model.write().overwrite().save("lr_model")
    predictions = model.transform(test_data)
    evaluator = MulticlassClassificationEvaluator(
    predictionCol="prediction", labelCol="label", metricName="accuracy"
        )
    print('Logistic regression accuracy: ',evaluator.evaluate(predictions))



    rfc = RandomForestClassifier()
    model=  rfc.fit(data)
    model.write().overwrite().save("rfc_model")
    predictions = model.transform(test_data)
    evaluator = MulticlassClassificationEvaluator(
    predictionCol="prediction", labelCol="label", metricName="accuracy"
        )
    print('Random Forest accuracy: ',evaluator.evaluate(predictions))


    gbt = GBTClassifier()
    model=  gbt.fit(data)
    model.write().overwrite().save("rfc_model")
    predictions = model.transform(test_data)
    evaluator = MulticlassClassificationEvaluator(
    predictionCol="prediction", labelCol="label", metricName="accuracy"
        )
    print('Gradientboost accuracy: ',evaluator.evaluate(predictions))

In [ ]:
train_data_func(train_df,test_df)

Logistic regression accuracy:  1.0
Random Forest accuracy:  1.0
Gradientboost accuracy:  1.0


In [ ]:
## Best Model is Logistic Regression

lr = LogisticRegression()
model = lr.fit(main_data)
model.write().overwrite().save("lr_model_best")
predictions = model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(
predictionCol="prediction", labelCol="label", metricName="accuracy"
    )
print('Logistic regression accuracy: ',evaluator.evaluate(predictions))

Logistic regression accuracy:  1.0
